In [1]:
import csv
from transformers import pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [2]:
data = "../Files/imdb_master.csv"
aux = {"neg": 0, "pos": 1}
texts = []
labels = []

with open(data, encoding="ISO-8859-1") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if not (row['label'] == "unsup"):
            texts.append(row['review'])
            labels.append(aux[row['label']])

texts = texts[int(len(texts) / 2) :]
labels = labels[int(len(labels) / 2) :]

In [3]:
temp_text, val_texts, temp_label, val_label = train_test_split(texts, labels, test_size=0.12, random_state=42)

In [4]:
def yield_data(texts):
    for text in texts:
        yield text

In [5]:
model_paths = ['cardiffnlp/twitter-roberta-base-sentiment', 
               'cardiffnlp/twitter-roberta-base-sentiment-latest', 
               'finiteautomata/bertweet-base-sentiment-analysis',
               'sbcBI/sentiment_analysis_model',
               'lxyuan/distilbert-base-multilingual-cased-sentiments-student', 
               'siebert/sentiment-roberta-large-english',
               'j-hartmann/sentiment-roberta-large-english-3-classes']

In [ ]:
model_preds = {}

for model_path in model_paths:
    model = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path, max_length=512, truncation=True, batch_size=500)

    preds = []
    for pred in model(yield_data(val_texts)):
        preds.append(pred)
    
    model_preds[model_path] = preds

In [ ]:
processedAux = {'LABEL_0': 0, 'LABEL_1': 1, 'LABEL_2': 2}
processedPreds = [processedAux[pred['label']] for pred in preds];
